In [ ]:
import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'RPOSE'))
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/core'))
sys.path.append(os.path.join(os.getcwd() + '/segmentation'))
import coloredlogs

import coloredlogs
coloredlogs.install()
from collections import OrderedDict
import time
import shutil
import datetime
import argparse
import signal
import yaml
import logging
from pathlib import Path
import copy
from src_utils import file_path, load_yaml, get_neptune_logger, get_tensorboard_logger
import datasets

env_cfg_path = os.path.join('cfg/env', os.environ['ENV_WORKSTATION_NAME']+ '.yml')
exp_cfg_path = "/home/jonfrey/RPOSE/cfg/exp/exp.yml"
exp = load_yaml(exp_cfg_path)
env = load_yaml(env_cfg_path)
test_dataloader = datasets.fetch_dataloader( exp['test_dataset'], env )
    


In [ ]:
model_point_list = test_dataloader.dataset._pcd_cad_list
base_path_list = test_dataloader.dataset._base_path_list
obj_idx_list = test_dataloader.dataset._obj_idx_list
camera_idx_list = test_dataloader.dataset._camera_idx_list

In [ ]:
from ycb.ycb_helper import get_bb_from_depth, get_bb_real_target
def get_init_pose_posecnn( obj_idx, p, h_gt, model_points, K, size):
  h_real_est = None
  base = "/home/jonfrey/PoseCNN-PyTorch/output/ycb_video/ycb_video_keyframe/vgg16_ycb_video_epoch_16.checkpoint.pth/"
  m = os.path.join( base, p.split('/')[-2] + '_' + p.split('/')[-1] +'.mat' )
  result = scio.loadmat(m)
  pcnn_class_idxs =  result['rois'][:,1]
  possible_rois_idx = np.where( pcnn_class_idxs == obj_idx )[0].tolist()
  target = model_points @ h_gt[:3,:3].T + h_gt[:3,3]
  bb = get_bb_real_target(torch.from_numpy( target[None,:,:] ), K[None])[0] 
  bb_gt = np.zeros( size ,dtype=bool)
  bb_gt[ int( bb.tl[0]): int( bb.br[0]), int( bb.tl[1] ): int( bb.br[1] )] = True # BINARY mask over BB
  
  if len( possible_rois_idx) > 0:
    # iterate over possible rois and find one with highest overlap
    overlaps = []
    for rois_idx in possible_rois_idx:
      bb_pcnn = np.zeros( size ,dtype=bool)
      tl = (result['rois'][rois_idx, 3], result['rois'][rois_idx, 2])
      br = (result['rois'][rois_idx, 5], result['rois'][rois_idx, 4])
      bb_pcnn[ int(tl[0]):int(br[0]),int(tl[1]):int(br[1])] = True # BINARY mask over BB
      overlaps.append(  (bb_gt * bb_pcnn).sum() / (bb_gt + bb_pcnn).sum() ) #IoU 
    
    max_rois = np.array( overlaps ).argmax()
    if overlaps[max_rois] > 0.5:
      # successfull selected
      matched_roi = possible_rois_idx[max_rois]
      quat = result['poses'][matched_roi,:4] #Nx7
      trans = result['poses'][matched_roi,-3:] #Nx7
      h_real_est = np.eye(4)
      h_real_est[:3,3] = trans
      h_real_est[:3,:3] = quat_to_rot( torch.from_numpy( quat)[None] , conv='wxyz', device='cpu').numpy()
  return h_real_est

In [ ]:
points = np.random.rand(1000, 2)   # 30 random points in 2-D
hull = ConvexHull(points)
hull.vertices.shape
points = points[hull.vertices]
# points.shape

def ccworder(A):
    A= A- np.mean(A, 1)[:, None]

from skimage.morphology import convex_hull_image
image = Image.fromarray( np.uint8(np.zeros( (300,400,3))) )


arr  = np.uint8(np.zeros( (300,400)))
arr[2:10,2:10] = 255
arr[130:140,120:130] = 255


res = convex_hull_image( arr )

display( Image.fromarray( arr  ) )
Image.fromarray( np.uint8( res )*255 )

In [ ]:
import scipy.io as scio
import numpy as np
base = "/home/jonfrey/PoseCNN-PyTorch/output/ycb_video/ycb_video_keyframe/vgg16_ycb_video_epoch_16.checkpoint.pth/"

In [ ]:
# %load /home/jonfrey/RPOSE/src/visu/visualizer.py
from PIL import ImageDraw
from scipy.spatial import ConvexHull
from skimage.morphology import convex_hull_image

from visu import Visualizer
visu = Visualizer(p_visu='/home/jonfrey/tmp',num_classes=21)

In [ ]:
import numpy as np
import scipy.io as scio
_, idxs = np.unique( np.array( base_path_list ),return_index=True )
idxs.tolist()

from PIL import Image

from ycb.rotations import *
import torch 
os.system( "mkdir /home/jonfrey/tmp/tracking_init_gt_correct_flow")

In [ ]:
for j,i in enumerate(idxs):
    p = base_path_list [i]
#     if j >1: break
    meta = scio.loadmat( p+"-meta.mat")
    img_store = np.array( Image.open(p+"-color.png") )
    K = test_dataloader.dataset.K[ str( camera_idx_list[i] ) ]
    print(j)
    img1 = copy.deepcopy( img_store )
    for k, obj_index in enumerate(meta['cls_indexes'].tolist()):
        H = np.eye(4)
        H[:3,:4] =  meta['poses'][:, :, k]   
        color = visu.SEG_COLORS[obj_index[0]-1].tolist()
        color[3] = 170
        color = tuple(color)
        img1 = visu.plot_estimated_pose( 
            img = img1, 
            points = model_point_list[obj_index[0]-1], 
            H = H,
            K = K, 
            tag = 'Test_init',
            color = color,
            w=0,
            not_log=True)
    
    img2 = copy.deepcopy( img_store )  
    for k, obj_index in enumerate(meta['cls_indexes'].tolist()):
      file = os.path.join( "/home/jonfrey/results/rpose/evaluate_pose/2021-04-24T10:19:18_tracking_with_gt_pose/", p[p.find("ycb/"):], f"{ obj_index[0]}.npy" )
      if os.path.isfile( file ):
            H = np.load( file )
            color = visu.SEG_COLORS[obj_index[0]-1].tolist()
            color[3] = 170
            color = tuple(color)
            img2 = visu.plot_estimated_pose( 
                img = img2, 
                points = model_point_list[obj_index[0]-1], 
                H = H,
                K = K, 
                tag = 'Test_init',
                color = color,
                w=0,
                not_log=True)
#     img3 = copy.deepcopy( img_store )  
#     base = "/home/jonfrey/PoseCNN-PyTorch/output/ycb_video/ycb_video_keyframe/vgg16_ycb_video_epoch_16.checkpoint.pth/"
#     m = os.path.join( base, p.split('/')[-2] + '_' + p.split('/')[-1] +'.mat' )
#     meta2 = scio.loadmat( m )
#     for k, obj_index in enumerate(meta['cls_indexes'].tolist()):  
#         h_gt = np.eye(4)
#         h_gt[:3,:4] =  meta['poses'][:, :, k]   
#         h_real_est = get_init_pose_posecnn( obj_index[0] , p, h_gt , model_point_list[obj_index[0]-1], K, (480,640) )
#         if h_real_est is not None:
#             color = visu.SEG_COLORS[obj_index[0]-1].tolist()
#             color[3] = 170
#             color = tuple(color)
#             img3 = visu.plot_estimated_pose( 
#                 img = img3, 
#                 points = model_point_list[obj_index[0]-1], 
#                 H = h_real_est,
#                 K = K, 
#                 tag = 'Test_init',
#                 color = color,
#                 w=0)

    space = np.zeros( (img1.shape[0],20,4) )
    res = np.concatenate( [ img2, space, img1],axis=1 )
    res = Image.fromarray( np.uint8(res) )
    res.save( f"/home/jonfrey/tmp/tracking_init_gt_correct_flow/{j:04d}_img.png" )    
#     display(res)

In [ ]:
img1.shape, img2.shape

In [ ]:
  # )  / ((K @ target.T)[:,2][:,None]) ).T 

In [ ]:
 #(K @ target.T)[2,:][None,:].repeat(2,0).shape
    
from PIL import ImageDraw, Image


In [ ]:
%%timeit

model_points = model_point_list[obj_index[0]-1] 
H, K
img = np.array( Image.open(p+"-color.png") )

color = (0,255,0,126)
base_layer = Image.fromarray( copy.deepcopy(img) ).convert("RGBA")
color_layer = Image.new('RGBA', base_layer.size, color=tuple( color[:3]) )
alpha_mask = Image.new('L', base_layer.size, 0)
alpha_mask_draw = ImageDraw.Draw(alpha_mask)

target = model_points @ H[:3,:3] + H[:3,3]
pixels = np.round(  ((K @ target.T)[:2,:] /  (K @ target.T)[2,:][None,:].repeat(2,0)).T )
_h,_w,_ = img.shape

w = 1
m = (pixels[:,0] >= w) * (pixels[:,1] >= w) * (pixels[:,1] < (_h-w-1)) * (pixels[:,0] < (_w-w-1))
pixels = pixels[m]

for u,v in pixels.tolist():
    alpha_mask_draw.ellipse( [(u - w, v - w ), (u + w + 1,v + w + 1) ], color[3])
base_layer = np.array( Image.composite(color_layer, base_layer, alpha_mask) )
#display(Image.composite(color_layer, base_layer, alpha_mask))
# 
#     return base_layer.astype(np.uint8)

In [ ]:
res = Image.fromarray( np.uint8(res) )
display(res)

In [ ]:
same = np.array( meta2_idx ) == np.array( obj_index )

In [ ]:
indexes = np.where(same)
indexes